# GlobalAIHub Derin Öğrenme Proje Kampı: Balık Sınıflandırma  #
#  **1. Kütüphanelerin yüklenmesi**
Gerekli kütüphanelerin import edilmesi 

In [64]:
import os
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.layers import Dense, Flatten, Input


# **2. Verinin yüklenmesi**
* fish_dir değişkeni, veri setinin bulunduğu dizinin yolunu tanımlar.
* data_classes, dizindeki alt klasör isimlerini alır (dosya uzantısı olmayanları filtreleyerek). Bu alt klasörler, farklı balık türlerini temsil eder.

In [65]:
# fish direction ile veri yükleme
fish_dir = '/kaggle/input/a-large-scale-fish-dataset/Fish_Dataset/Fish_Dataset'
# dosya adında nokta olmayan verileri filtreleme 
data_classes = [filename for filename in os.listdir(fish_dir) if '.' not in filename]
print(data_classes)

['Hourse Mackerel', 'Black Sea Sprat', 'Sea Bass', 'Red Mullet', 'Trout', 'Striped Red Mullet', 'Shrimp', 'Gilt-Head Bream', 'Red Sea Bream']


# 3. Etiket ve Yol Listesinin Oluşturulması
* label ve path listeleri tanımlanır.
* os.walk, belirtilen dizindeki tüm dosyaları ve alt dizinleri gezerek görüntülerin yollarını ve etiketlerini toplar.
* .png uzantılı dosyalar filtrelenir ve sadece gerekli dizinler (örneğin, 'GT' olmayanlar) kullanılarak etiketler ve dosya yolları eklenir.

In [66]:
# etiket listesinin oluşturulması 
label = []
# yol listesinin oluşturulması
path = []
# tüm dizin ve dosyaları dolaşarak görsellerin yollarını ve etiketlerini listelere eklenmesi
for dir_name, _, filenames in os.walk(fish_dir):
    for filename in filenames:
        # görüntülerin eklenmesi 
        if os.path.splitext(filename)[-1]== ".png":
            # dizin adı 'GT' içermiyorsa (bu dizindeki veriler kullanılmıyor)
            if dir_name.split()[-1]!= 'GT':
                 # etiket eklenmesi
                label.append(os.path.split(dir_name)[-1])
                # dosya yolunun eklenmesi
                path.append(os.path.join(dir_name, filename))
                

# 4. DataFrame Oluşturulması

In [67]:
# görüntü yolları ve etiketlerin bulunduğu DataFrame oluşturulması
df = pd.DataFrame(columns= ['path', 'label'])
df['path'] = path
df['label'] = label

df
               

,path,label
0,/kaggle/input/a-large-scale-fish-dataset/Fish_...,Hourse Mackerel
1,/kaggle/input/a-large-scale-fish-dataset/Fish_...,Hourse Mackerel
2,/kaggle/input/a-large-scale-fish-dataset/Fish_...,Hourse Mackerel
3,/kaggle/input/a-large-scale-fish-dataset/Fish_...,Hourse Mackerel
4,/kaggle/input/a-large-scale-fish-dataset/Fish_...,Hourse Mackerel
...,...,...
8995,/kaggle/input/a-large-scale-fish-dataset/Fish_...,Red Sea Bream
8996,/kaggle/input/a-large-scale-fish-dataset/Fish_...,Red Sea Bream
8997,/kaggle/input/a-large-scale-fish-dataset/Fish_...,Red Sea Bream
8998,/kaggle/input/a-large-scale-fish-dataset/Fish_...,Red Sea Bream


# 5. Keşifçi Veri Analizi (Exploratory Data Analysis — EDA)
Veriler üzerindeki ilk gözlemlerin yapılabilmesi için uygulanır.

In [68]:
def check_df(dataframe):
    print("##################### Shape #####################")
    print(dataframe.shape)
    print("##################### Types #####################")
    print(dataframe.dtypes)
    print("##################### Head #####################")
    print(dataframe.head(3))
    print("##################### Tail #####################")
    print(dataframe.tail(3))
    print("##################### NA #####################")
    print(dataframe.isnull().sum())
    print("##################### INFO #####################")
    print(dataframe.info())


check_df(df)

##################### Shape #####################
(9000, 2)
##################### Types #####################
path     object
label    object
dtype: object
##################### Head #####################
                                                path            label
0  /kaggle/input/a-large-scale-fish-dataset/Fish_...  Hourse Mackerel
1  /kaggle/input/a-large-scale-fish-dataset/Fish_...  Hourse Mackerel
2  /kaggle/input/a-large-scale-fish-dataset/Fish_...  Hourse Mackerel
##################### Tail #####################
                                                   path          label
8997  /kaggle/input/a-large-scale-fish-dataset/Fish_...  Red Sea Bream
8998  /kaggle/input/a-large-scale-fish-dataset/Fish_...  Red Sea Bream
8999  /kaggle/input/a-large-scale-fish-dataset/Fish_...  Red Sea Bream
##################### NA #####################
path     0
label    0
dtype: int64
##################### INFO #####################
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9

In [69]:
df["label"].value_counts()


label
Hourse Mackerel       1000
Black Sea Sprat       1000
Sea Bass              1000
Red Mullet            1000
Trout                 1000
Striped Red Mullet    1000
Shrimp                1000
Gilt-Head Bream       1000
Red Sea Bream         1000
Name: count, dtype: int64

# 6. Görüntüleri Yükleme ve Boyutlandırma
* load_and_resize_images fonksiyonu, verilen dosya yollarından görüntüleri yükler ve belirli bir boyuta (128x128) yeniden boyutlandırır.
* Görüntülerin piksel değerleri 0 ile 1 arasında olacak şekilde normalleştirilir, bu da modelin eğitimini iyileştirir.
* Fonksiyon, boyutlandırılmış görüntülerin bir numpy dizisini döndürür.

In [70]:
# Görüntüleri yükleme ve boyutlandırma fonksiyonu (örneğin 128x128 boyutunda)
def load_and_resize_images(paths, img_size=(128, 128)):
    images = []
    for img_path in paths:
        img = cv2.imread(img_path)
        img = cv2.resize(img, img_size)
        img = img / 255.0  # Normalizasyon
        images.append(img)
    return np.array(images)


# 7. Veri setinin Train ve Test olarak ayrılması
* Veri seti train ve test olmak üzere ikiye ayrılır. Model train set ile eğitilir, test set ile test edilir. 
* train_test_split, DataFrame'i eğitim ve test setlerine ayırır. Burada %80 eğitim, %20 test verisi kullanılır. random_state ise sonuçların tekrarlanabilirliğini sağlar.

In [71]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# 8. Görüntülerin Yüklenmesi
Eğitim ve test setleri için görüntüler, yukarıda tanımlanan load_and_resize_images fonksiyonu ile yüklenir ve boyutlandırılır. Sonuç olarak X_train ve X_test numpy dizileri oluşturulur.

In [72]:
# Eğitim ve test verilerinin yüklenmesi ve boyutlandırılması
X_train = load_and_resize_images(train_df['path'])
X_test = load_and_resize_images(test_df['path'])

# 9. Etiketlerin Dönüştürülmesi
LabelEncoder, etiketleri sayısal formatta dönüştürmek için kullanılır. Bu işlem, her balık türünün bir tamsayı ile temsil edilmesini sağlar.

In [73]:
# LabelEncoder ile etiketlerin dönüştürülmesi
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(train_df['label'])
y_test = label_encoder.transform(test_df['label'])

# 10. One-Hot Encoding
* num_classes, toplam sınıf sayısını belirler.
* to_categorical, etiketleri one-hot encoding formatına dönüştürür. Bu, her sınıfın bir vektörde temsil edilmesini sağlar.

In [74]:
# Sınıf etiketlerini one-hot encoding yapma
num_classes = len(label_encoder.classes_)  # Sınıf sayısını belirleme
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

# 11. Model Oluşturma
* Model Tanımı: Sequential modeli, katmanları sıralı bir şekilde oluşturmak için kullanılır.
* Input Katmanı: Görüntü boyutları burada tanımlanır. Bu katman, modelin girdi boyutunu belirler.
* Flatten Katmanı: Görüntüleri tek boyutlu bir vektöre dönüştürür.
* Dense Katmanları: İki adet yoğun katman eklenir. Bu katmanlar, sırasıyla 128 ve 64 nörona sahip ve relu aktivasyon fonksiyonunu kullanır.
* Çıkış Katmanı: num_classes kadar nöron içerir ve softmax aktivasyon fonksiyonu ile her sınıf için olasılık tahmini yapar.

In [75]:
# Model oluşturma
model = Sequential()

# İlk katman olarak Input eklenmesi
model.add(Input(shape=(128, 128, 3)))

# Görüntü boyutları 128x128x3 (RGB görüntü)
model.add(Flatten())  # input_shape'yi Flatten katmanına vermeye gerek yok

# İlk yoğun (dense) katman
model.add(Dense(128, activation='relu'))

# İkinci yoğun katman
model.add(Dense(64, activation='relu'))

# Çıkış katmanı: Her sınıf için tahmin (num_classes kadar nöron)
model.add(Dense(num_classes, activation='softmax'))

# 12. Modeli Derleme
* Compile: Modelin eğitimine başlamadan önce derlenmesi gerekir.
* Optimizer: Adam optimizasyon algoritması kullanılır.
* Loss Function: Categorical crossentropy kayıp fonksiyonu, çok sınıflı sınıflandırma problemleri için uygundur.
* Metrics: Modelin başarımını değerlendirmek için doğruluk metriği kullanılır.

In [76]:
# Modeli derleme
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 13. Modeli Eğitme
* fit() Metodu: Modelin eğitim verileri üzerinde eğitilmesini sağlar.
* epochs: Modelin verilerle kaç kez eğitileceğini belirler (10 kez).
* batch_size: Her iterasyonda kullanılacak veri miktarıdır (16).
* validation_data: Modelin doğruluk oranını değerlendirmek için test verileri sağlanır.

In [77]:
# Modeli eğitme
model.fit(X_train, y_train, epochs=10, batch_size=16, validation_data=(X_test, y_test))


Epoch 1/10
450/450 ━━━━━━━━━━━━━━━━━━━━ 23s 49ms/step - accuracy: 0.3490 - loss: 3.5345 - val_accuracy: 0.6617 - val_loss: 0.9831
Epoch 2/10
450/450 ━━━━━━━━━━━━━━━━━━━━ 21s 46ms/step - accuracy: 0.6629 - loss: 0.9786 - val_accuracy: 0.5950 - val_loss: 1.1445
Epoch 3/10
450/450 ━━━━━━━━━━━━━━━━━━━━ 21s 47ms/step - accuracy: 0.6848 - loss: 0.9132 - val_accuracy: 0.7589 - val_loss: 0.7072
Epoch 4/10
450/450 ━━━━━━━━━━━━━━━━━━━━ 33s 74ms/step - accuracy: 0.7740 - loss: 0.6595 - val_accuracy: 0.8183 - val_loss: 0.5402
Epoch 5/10
450/450 ━━━━━━━━━━━━━━━━━━━━ 29s 64ms/step - accuracy: 0.8358 - loss: 0.4774 - val_accuracy: 0.6867 - val_loss: 0.9776
Epoch 6/10
450/450 ━━━━━━━━━━━━━━━━━━━━ 21s 47ms/step - accuracy: 0.8262 - loss: 0.5277 - val_accuracy: 0.7861 - val_loss: 0.6229
Epoch 7/10
450/450 ━━━━━━━━━━━━━━━━━━━━ 20s 45ms/step - accuracy: 0.8690 - loss: 0.3750 - val_accuracy: 0.8094 - val_loss: 0.5623
Epoch 8/10
450/450 ━━━━━━━━━━━━━━━━━━━━ 21s 46ms/step - accuracy: 0.8820 - loss: 0.3483 - 

# 14. Modelin Değerlendirilmesi
* evaluate() Metodu: Test verileri kullanılarak modelin kaybı ve doğruluğu hesaplanır.
* Sonuçlar konsola yazdırılır. score[0] kaybı, score[1] ise doğruluğu temsil eder

In [78]:
# Modeli değerlendirme
score = model.evaluate(X_test, y_test)
print(f'Test kaybı: {score[0]}')
print(f'Test doğruluğu: {score[1]}')

57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.8545 - loss: 0.4201
Test kaybı: 0.368929386138916
Test doğruluğu: 0.8722222447395325
